# Howto

F. Dubois

First you need to build a gmsh binary with the gmshpy module.  
First download the code (login gmsh, password gmsh):  
*svn co https://onelab.info/svn/gmsh/trunk gmsh*  
Then you have to build the code; be carefull many dependencies are necessary/possible (fltk, etc). If you have already installed gmsh with your favorite package manager (port, apt-get, yum, etc) they should be installed.  
To compile the python module :  
*mkdir build; cd build*  
*cmake ../gmsh -DENABLE_WRAP_PYTHON=1*  
*make -j4*  
Be carefull that all some cmake variables are well defined: PYTHON_EXECUTABLE, PYTHON_LIBRARY, PYTHON_INCLUDE_DIR, etc  
If compilation works fine you have to add the path to the gmshpy module :  
*export PYTHONPATH=$PYTHONPATH:blabla/build/wrappers*

## Creating a geometrical model :  

In [ ]:
from gmshpy import *
import os

# from http://en.wikipedia.org/wiki/Constructive_solid_geometry

R = 1.4
s = .7
t = 1.25

myModel = GModel()
myModel.addBlock([-R,-R,-R],[R,R,R])

myTool = GModel()
myTool.addSphere(0,0,0,R*t)

myModel.computeBooleanIntersection(myTool)

# building an intermediate complicated structure
myTool2 = GModel()
myTool2.addCylinder([-2*R,0,0],[2*R,0,0],R*s)

myTool3 = GModel()
myTool3.addCylinder([0,-2*R,0],[0,2*R,0],R*s)

myModel2 = GModel()
myModel2.addCylinder([0,0,-2*R],[0,0,2*R],R*s)
myModel2.computeBooleanUnion(myTool2)
myModel2.computeBooleanUnion(myTool3)
myModel2.setVisibility(0)

myModel.computeBooleanDifference(myModel2)

#
myModel.setAsCurrent()
myModel.setVisibility(1)

if not os.path.isdir('./output'):
  os.mkdir('./output')

myModel.save("./output/wikipedia.brep")
myModel.writeGEO("./output/wikipedia.geo")

# meshing
#GmshSetOption("Mesh", "CharacteristicLengthFactor", 0.4);
#myModel.mesh(3);
#myModel.save("./output/wikipedia.msh");

# Drawing
#FlGui.instance();
#FlGui.run();


## Creating a 2D mesh :

In [ ]:
from gmshpy import *

lc = 0.5
GmshSetOption('Mesh', 'CharacteristicLengthFactor', lc)
 
g = GModel()

# creating geometry
v1 = g.addVertex(0, 0, 0, lc)
v2 = g.addVertex(1, 0, 0, lc)
v3 = g.addVertex(1, 1, 0, lc)
v4 = g.addVertex(0, 1, 0, lc)
e1 = g.addLine(v2, v1)
e2 = g.addLine(v3, v2)
e3 = g.addLine(v4, v3)
e4 = g.addLine(v4, v1)

v11 = g.addVertex(.4, .4, 0, lc)
v12 = g.addVertex(.6, .4, 0, lc)
v13 = g.addVertex(.6, .5, 0, lc)
v14 = g.addVertex(.4, .6, 0, lc)
e11 = g.addLine(v11, v12)
e12 = g.addLine(v12, v13)
e13 = g.addLine(v13, v14)
e14 = g.addLine(v14, v11)

f = g.addPlanarFace([[e1,e2,e3,e4],[e11,e12,e13,e14]])

if not os.path.isdir('./output'):
  os.mkdir('./output')

g.writeGEO("./output/square.geo")

# meshing
g.mesh(2)
g.save("./output/square.msh")

# Drawing
#FlGui.instance()
#FlGui.run()


## Creating a 3D mesh 

In [ ]:
from gmshpy import *

GmshSetOption('Mesh', 'CharacteristicLengthFactor', 0.01)

myModel = GModel();

R = 0.3
myModel.addSphere(0.5,0.5,0.5,R)

myModel.setAsCurrent()

if not os.path.isdir('./output'):
  os.mkdir('./output')

myModel.writeGEO("./output/sphere.geo")

myModel.mesh(2);
myModel.save("./output/sphere.msh")

# Drawing
#FlGui.instance()
#FlGui.run()


## Remeshing from a mesh

In [ ]:
from gmshpy import *

GmshSetOption('General', 'Verbosity', 99.0) 
GmshSetOption('Mesh', 'SaveAll', 1.0) 
GmshSetOption('Mesh', 'Algorithm', 1.0)

g =  GModel()
g.setFactory('Gmsh')

g.load("./input/aneurysm.stl")
g.createTopologyFromMesh()

face = g.getFaceByTag(1)
newent  = g.extrudeBoundaryLayer(face, 4, 0.5, 0, -1)
newent2 = g.extrudeBoundaryLayer(face, 4, -0.5, 1, -1)

print "*** face = %d " % face.tag()
print "*** new face = %d newface2 = %d " % (newent[0].tag(), newent2[0].tag())

g.mesh(2)

if not os.path.isdir('./output'):
  os.mkdir('./output')

g.save("./output/aneurysmBL.msh")

# Drawing
#FlGui.instance()
#FlGui.run()
